In [2]:
import pandas as pd
import ast as ast
import numpy as np
from typing import *

from task_2_helpers import *

### Data Read

In [6]:
# df_business = pd.read_parquet('data/ATML2024_businesses.parquet')
df_business = pd.read_csv('data/ATML2024_businesses.csv', header=0)

# df_Train_reviews = pd.read_parquet('data/ATML2024_reviews_train.parquet')
df_Train_reviews = pd.read_csv('data/ATML2024_reviews_train.csv', header=0)

# # df_users = pd.read_parquet('data/ATML2024_users.parquet')
# df_users = pd.read_csv('data/ATML2024_users.csv', header=0, nrows=100)

### Cleanup business data columns

In [7]:
# Convert dictionary columns to the correct types @@@@ Run only once
df_business = df_business[df_business['city'].isin(bussiness_city)]
df_business['attributes'] = df_business['attributes'].apply(lambda x: ast.literal_eval(x) if not pd.isna(x) or not pd.isnull(x)  else dict())
df_business['hours'] = df_business['hours'].apply(lambda x: ast.literal_eval(x) if not pd.isna(x) or not pd.isnull(x)  else dict())

# df_business['weekend'] = df_business['hours'].apply(lambda x: extract_weekend_value(x))
df_business['hours_list'] = df_business['hours'].apply(lambda x: extract_hours_value(x))
df_business['attributes_list'] = df_business['attributes'].apply(lambda x: sorted(extract_attributes_value(x)))
df_business['categories_list'] = df_business['categories'].apply(lambda x: sorted(map(lambda a: a.strip().lower(),  x.split(',')) ) if isinstance(x, str) else [])



### Join Business Data With Ratings Data and Conduct Exploratory Data Analysis (EDA)

In [8]:
df_business_data = df_business[['business_id', 'postal_code', 'attributes_list', 'hours_list', 'categories_list']]
df_ratings_data = df_Train_reviews[['business_id', 'rating', 'useful', 'funny', 'cool', 'text']]

df_data = pd.merge(df_business_data, df_ratings_data, left_on= 'business_id', right_on='business_id')
df_data.head()

,business_id,postal_code,attributes_list,hours_list,categories_list,rating,useful,funny,cool,text
0,wm9eoqjytVbC7dQcM4WSTM,19107,"[Alcohol, BikeParking, BusinessAcceptsCreditCa...","[[7.0, 20.0], [7.0, 20.0], [7.0, 20.0], [7.0, ...","[bakeries, bubble tea, coffee & tea, food, res...",4,1,0,2,So I don't know how the other review was doubl...
1,wm9eoqjytVbC7dQcM4WSTM,19107,"[Alcohol, BikeParking, BusinessAcceptsCreditCa...","[[7.0, 20.0], [7.0, 20.0], [7.0, 20.0], [7.0, ...","[bakeries, bubble tea, coffee & tea, food, res...",5,0,0,0,I love this place.. I love their chunky fried...
2,wm9eoqjytVbC7dQcM4WSTM,19107,"[Alcohol, BikeParking, BusinessAcceptsCreditCa...","[[7.0, 20.0], [7.0, 20.0], [7.0, 20.0], [7.0, ...","[bakeries, bubble tea, coffee & tea, food, res...",5,0,0,1,I ordered so many cake from here for our famil...
3,wm9eoqjytVbC7dQcM4WSTM,19107,"[Alcohol, BikeParking, BusinessAcceptsCreditCa...","[[7.0, 20.0], [7.0, 20.0], [7.0, 20.0], [7.0, ...","[bakeries, bubble tea, coffee & tea, food, res...",4,0,0,0,Ordered a birthday cake here for the first tim...
4,wm9eoqjytVbC7dQcM4WSTM,19107,"[Alcohol, BikeParking, BusinessAcceptsCreditCa...","[[7.0, 20.0], [7.0, 20.0], [7.0, 20.0], [7.0, ...","[bakeries, bubble tea, coffee & tea, food, res...",5,1,0,0,This is my favorite bakery in Chinatown! It's ...
...,...,...,...,...,...,...,...,...,...,...
144869,QaNNUNpPIvqk6LH1QjG4JL,19103,"[AcceptsInsurance, BikeParking, BusinessAccept...","[[7.0, 20.0], [7.0, 20.0], [7.0, 20.0], [7.0, ...","[active life, doctors, fitness & instruction, ...",3,0,0,0,I've been to a few studios in the city and I c...
144870,QscEk0TPjLIlLgQzIN9TnW,19147,"[Alcohol, Ambience, BikeParking, BusinessAccep...","[[11.0, 22.0], [11.0, 22.0], [-1, -1], [11.0, ...","[mexican, restaurants]",5,2,1,1,First time trying this restaurant and I had a ...
144871,QscEk0TPjLIlLgQzIN9TnW,19147,"[Alcohol, Ambience, BikeParking, BusinessAccep...","[[11.0, 22.0], [11.0, 22.0], [-1, -1], [11.0, ...","[mexican, restaurants]",5,1,0,0,Amazing food and wonderful service! We got the...
144872,QscEk0TPjLIlLgQzIN9TnW,19147,"[Alcohol, Ambience, BikeParking, BusinessAccep...","[[11.0, 22.0], [11.0, 22.0], [-1, -1], [11.0, ...","[mexican, restaurants]",5,1,0,0,This hidden gem in south philly is why we need...


In [9]:
# df_data.to_parquet('data/ATML2024_Task2_PhiliBussRatings.parquet')
# print(df_data.shape)

(145048, 10)


### Explanatory Data Analysis

In [27]:
df_data = pd.read_parquet('data/ATML2024_Task2_PhiliBussRatings.parquet', engine='pyarrow')
print(df_data.shape)

(145048, 10)


### EDA

In [16]:
all_categorizes = set(np.concatenate(df_data.categories_list.values).ravel()) #df_data.categories_list.values.flatten()

In [21]:
rating_1_cat = set(np.concatenate(df_data[df_data.rating == 1].categories_list.values).ravel())
rating_2_cat = set(np.concatenate(df_data[df_data.rating == 2].categories_list.values).ravel())
rating_3_cat = set(np.concatenate(df_data[df_data.rating == 3].categories_list.values).ravel())
rating_4_cat = set(np.concatenate(df_data[df_data.rating == 4].categories_list.values).ravel())
rating_5_cat = set(np.concatenate(df_data[df_data.rating == 5].categories_list.values).ravel())

In [ ]:
x = 2